# Lecture 4: Loss Functions and Regularization


In this notebook, you'll find various tasks encompassing both theoretical and coding exercises. Each exercise corresponds to a specific number of points, which are explicitly indicated within the task description.

Always use the Jupyter kernel associated with the dedicated environment when compiling the notebook and completing your exercises.

## Excercise 1 (Theory) (25/100)

### Regularization techniques

In the lecture you heard about different types of regularization techniques which help the training of a neural network (or a machine learning model in general) to generalize better.     
First, write down the formulae for the $L_1$ and $L_2$ type of regularization and for a given loss function $\mathcal{L}$ of a regression model write down its regularized version.
Then for each of those regularization techniques, elaborate on the following aspects:        
- **Task (1.a)** **(5 pts.)** What are the effect of $L_1$($L_2$) regularization on the model's weights $W$?
- **Task (1.b)** **(5 pts.)** How robust is $L_1$($L_2$) regularization with respect to outliers data?
- **Task (1.c)** **(5 pts.)** Which one of the two techniques is more computationally advantageous and why?
- **Task (1.d)** **(5 pts.)** Which one of the two techniques is more suited for feature selection?


> Hint: by *feature selection (extraction)* we mean the ability of selecting the most important and informative features from the available set of features. In many real-world datasets, there can be a large number of features, some of which may be irrelevant, redundant, or noisy. Including all features in the model which can induce fitting the *noise* in the training data rather than capturing the underlying patterns. Feature selection thus aims to mitigate this by selecting only the **most relevant** features, which can improve model performance, reduce complexity, and enhance interpretability.


- **Task (1.e)** **(5 pts.)** Based on what you have learned in the lecture and further readings you may have done, discuss why one type of regularization can be more performant than the other. Based on this, in which scenarios would you recommend using one over the other?


The diffrent regularization are given by
$$
\mathcal{L}_{\text{reg, }L_1} = \mathcal{L} + \lambda \sum_{i=1}^{n} |w_i|
$$
and
$$
\mathcal{L}_{\text{reg, }L_2} = \mathcal{L} + \lambda \sum_{i=1}^{n} w_i^2
$$

- Task (1.a): The $L_1$ regularization pushes most weights to zero.  This is due to the modulus of the weight. On the other hand the $L2$ regularization scales quadratic with the weight, leading to less penalization for small terms. Therfore the regularization encourages smaller weights, without forcing them to 0 ( for reasonable choices of $\lambda$).

- Task(1.b): The robustness against outliers follows directly from Task(1.a). Since $L_1$ pushes most weights to zero, it is robust against outliners. $L_2$ only enforces only small weights, so in principle outliniers are still a problem.

- Task(1.c)
$L_1$ is more advantagous for multiple reasons. First $L_1$ produces sparse tensor, which reduces the computational complexity by reducing the number of operations. Second the implementation of the modules can be realized using bitwise operations which are usually faster than floating point arithmetic (although this is probably neglactible).

- Task(1.d)
The $L_1$ regularization is more suited because it removes features completly by setting the associated weights to zero.

- Task(1.d)
The $L_1$ regularization is useful when the dataset has many redundent or irrelevant features, it can efficently eliminate them. Furthermore the $L_1$ regularization is useful when teh DataSet is noisy or have features which count as outliner features. The $L_2$ regularization is useful when the dataset consists of many features, which have roughly the same contribution. A nother realavent point is that the convergence while training should be higher for $L_2$ regularization. From a pure performance standpoint the $L_1$ regularization seems as the better bet. In most cases, however, it depends on the exact task.

> #### Your solution here

## Excercise 2 (Theory) (10/100)

### Neural Network Generalization

To enable a neural network to generalize effectively with limited data, it's beneficial to ensure it's robust against small local variations. Achieving this involves constraining the gradient norm $\vert\frac{\partial f}{\partial x}\vert$ across all input values $x$ within the domain. Given the potentially high dimensionality of the input domain, directly minimizing the gradient norm is impractical. Instead, we opt to minimize an upper bound on it, which solely relies on the model parameters.


We instantiate a neural network comprising two layers, featuring $d$ input neurons, $h$ hidden neurons, and one output neuron. Let $W$ represent the weight matrix with dimensions $d \times h$, and $(b_j)_{j=1}^h$ denote a set of biases. We use $W_{i,:}$ to refer to the $i$-th row of the weight matrix and $W_{:,j}$ for its $j$-th column. The neural network performs the following computation:
$$
\begin{align}
a_j &= \max(0,W_{:,j}^\top x + b_j) & \text{(layer 1)}\\
f(x) &= \textstyle \sum_j a_j & \text{(layer 2)}
\end{align}
$$

The first layer detects patterns of the input data, and the second layer performs a pooling (i.e., sum) operation over these detected patterns.

Show that the gradient norm of the network can be upper-bounded as:
$$
\Big\|\frac{\partial f}{\partial x}\Big\| \leq \sqrt{h} \cdot \|W\|_F
$$

> Suggestion: You can use the Cauchy-Schwarz inequality

> #### Your solution here

## Excercise 1 (Programming) (20/100)

### Optimization

This first part contains some test-loss surfaces and a complete training loop, to give a play ground for testing out different optimizers.
The code is based on torch and you can find the documentation of the optimizers here: [https://pytorch.org/docs/stable/optim.html](https://pytorch.org/docs/stable/optim.html).        


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch 

In the following, three different loss functions are provided:

In [4]:
def loss1(xy):
    return (xy**2).sum(-1)/10.8822

def loss2(xy):
    return ((5 * xy[:,0]**2 + 0.3 * xy[:, 1]**2)/28.8377)

def loss3(xy):
    w = -20, 9, -12, 4, -3, -0.7, -0.5, 0.5, 0.3
    x, y = xy[:, 0]*2, xy[:, 1]*3
    return (w[0] * x + w[1] * y + w[2] * x * y + w[3] * x**2 + w[4] * y **2 + 
            w[5] * x**3 + w[6] * y**3 + 0.3 * (x**4 + y**4 - 0.99 * y**2 * x**2) +
           torch.cos(0.3 * x + y * 0.7) * 37 + torch.sin(0.5 * y) * 19)/1175.0842

Alongside with an example training function `train_example`

In [5]:
class ML_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.params = torch.nn.Parameter(torch.tensor(((-1.5, -2),)))


class Batch_loss():
    def __init__(self, loss, bs=0):
        self.bs = bs
        self.loss = loss
        torch.manual_seed(0)
    
    def __call__(self, params):
        loss = self.loss(params)
        if self.bs > 0:
            n_noise = 40
            x = torch.normal(0, 4, (n_noise, 2))
            l = torch.normal(0, np.sqrt(100 / self.bs), (n_noise,))
            ls = torch.normal(0., 1, (n_noise,))

            dists = ((-torch.cdist(params.unsqueeze(0), x.unsqueeze(0))/ls.abs()).exp() * l).sum(-1).squeeze(0)
            loss +=  dists
        return loss


def train_example(loss_func,  optim, optim_kwargs, bs=0, nsteps = int(5e2)):  
    model = ML_Model()
    optim_ = optim(model.parameters(), **optim_kwargs)
    param_traj = [list(model.params[0].detach().cpu().numpy())]
    
    loss = Batch_loss(loss_func, bs=bs)
    for i in range(nsteps):
        train_loss = loss(model.params)
        train_loss.backward()
        optim_.step()
        model.zero_grad()
        param_traj.append(list(model.params[0].detach().cpu().numpy()))
        #if ((i + 1) % max(nsteps//10, 1)) == 0:
        #    print(f"Epoch {i + 1} trainloss {train_loss.item():.2e}")
    bs_str = f"batch-size: {bs}" if bs else "full-batch"
    splitter = "'"
    plt.plot(*np.asarray(param_traj).T,  
                        alpha=0.9, label=f'{str(optim).split(splitter)[1]}: {optim_kwargs} {bs_str}')
    
    print(f'{str(optim).split(splitter)[1]}: {optim_kwargs} {bs_str}||loss = {loss_func(model.params).item():.2f}')
    return model.params

In [6]:
def plot_loss_surface_toy(loss_func):
    l = 4
    x = np.linspace(-l, l, 100)
    y = np.linspace(-l, l, 100)
    X, Y = np.meshgrid(x, y)
    fig = plt.figure()
    
    loss = Batch_loss(loss_func, bs=0)    
    Z = loss(torch.stack([torch.tensor(x.flatten()).float() for x in [X, Y]]).T).reshape(X.shape)
    plt.contourf(X, Y, Z, cmap='viridis', levels=25)
    plt.ylim(-l,l)
    plt.xlim(-l,l)

This training function takes as input the following arguments
```python
train_example(
    loss_func: function, # loss function
    optim: type, # optimizer
    optim_kwargs: dict, # dictionary with optimizer hyperparameters
    bs=0: int # batch size
)
```
while the `plot_loss_surface` function provides you with an example of how to plot the loss surface of a given loss function. 

You can combine all these ingredients together to get a minimal working example.
In particular, you are asked to:
- **Task (1.a)** **(10 pts.)** play around with the different loss functions and hyperparameters. From what you have learned in the lecture, try to come up with different scenarios, e.g., choices of hyperparameters, which lead to *good* and *bad* outcomes. 
- **Task (1.b)** **(10 pts.)** plot the results and comment on them. 

In [23]:
# ----------------------------------
#  Your code here
# ----------------------------------

## Excercise 2 (Programming) (45/100)

### Regularization

This excercise will let you apply some of the concepts discussed in the lecture and in the theory part above. First and foremost, you will need to install `sklearn` as a new package in your environment.
To do this, make sure to be in the folder where your virtual environment is and do

```bash
$ source venv-name/bin/pip3 install sklearn
```

if this has worked correctly, you can reload the kernel of the jupyter notebook and you should be able to run

```python
import sklearn
```

Should this command run without any problem (try it out in a separate python cell) you can proceed with the task!

The following code extracts the [Breast Cancer Wisconsin Dataset](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data/download?datasetVersionNumber=2&ref=hackernoon.com) such that is already partitioned into training and test data. The data is normalized such that each dimension has mean 0 and variance 1.

In [8]:
import numpy, numpy.random

import sklearn,sklearn.datasets

def cancer_data():
    
    D = sklearn.datasets.load_breast_cancer()
    X = D['data']
    T = D['target']

    # Partition the data
    N = len(X)
    perm = numpy.random.mtrand.RandomState(1).permutation(N)
    Xtrain,Xtest = X[perm[:N//2]],X[perm[N//2:]]
    Ttrain,Ttest = T[perm[:N//2]],T[perm[N//2:]]
    print(len(Xtrain), len(Ttrain))
    # Normalize input data
    m,s = Xtrain.mean(axis=0),Xtrain.std(axis=0)+1e-9
    for x in Xtrain,Xtest: 
        x -= m; x /= s

    # Normalize targets
    m,s = Ttrain.mean(),Ttrain.std()+1e-9
    for t in Ttrain,Ttest: 
        t=t.astype(float)
        t -= m; t /= s

    return Xtrain,Ttrain,Xtest,Ttest


Xtrain,Ttrain,Xtest,Ttest = cancer_data()

nx = Xtrain.shape[1]
nh = 100

284 284


The following code implements a class `NeuralNetworkRegressor`. This is a neural network with a single hidden layer of width `nh` and a `ReLU` activation function. 
The function `reg` is a regularizer which we set initially to zero (i.e. no regularizer). 
Because the dataset is small, the network is optimized in batch mode, using the Adam optimizer. 

> Note: The `Adam` otpimizer is an efficient optimizer widely used in Machine Learning. You'll learn more about it in the next lecture and you'll be using it in the next excercise as the default optimizer of the given `fit` function. If you want to know more about Adam and momentum based optimizer you can check [this link](https://medium.com/@vinodhb95/momentum-optimizer-6023aa445e18)

In [9]:
import numpy,torch,sklearn,sklearn.metrics
from torch import nn,optim

class NeuralNetworkRegressor:

    def __init__(self):
        
        torch.manual_seed(0)
        
        self.model = nn.Sequential(nn.Linear(nx,nh),nn.ReLU())
        self.pool  = lambda y: 0.1*(y[:,:nh//2].sum(dim=1)-y[:,nh//2:].sum(dim=1))
        self.loss  = nn.MSELoss()

    def reg(self): 
        return 0
        
    def fit(self,X,T,nbit=10000):
        
        X = torch.Tensor(X)
        T = torch.Tensor(T)

        optimizer = optim.Adam(self.model.parameters(),lr=0.01)
        for _ in range(nbit):
            optimizer.zero_grad()
            (self.loss(self.pool(self.model(X)),T)+self.reg()).backward()
            optimizer.step()
                
    def predict(self,X):
        return self.pool(self.model(torch.Tensor(X)))

    def score(self,X,T):
        return sklearn.metrics.r2_score(T,numpy.array(self.predict(X).data)) 

> N.B. The `score` function above implements the [Coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) ($R^2$ score). It determines how well a model predicts an outcome and it is bounded by 1 (e.g. perfect predictor). It is unbounded from elow as a model can, in general, be arbitrarily bad. 

#### **Task (2.a)** **(5 pts.)**
 Try to instantiate a neural network from the class above and train it. Then test its performance on test data. What do you conclude?

In [22]:
# ----------------------------------
#  Your code here
# ----------------------------------

#### **Task (2.b)** **(25 pts.)** 

We want to use regularization towards improving the neural network model. We consider the following two quantities:
- $\|W\|_\text{Frob} =  \sqrt{\sum_{i=1}^d \sum_{j=1}^h  w_{ij}^2}$
- $\text{Grad} = \textstyle \frac1N \sum_{n=1}^N\|\nabla_{\boldsymbol{x}}f (\boldsymbol{x_n})\|$

Here $d$ is the number of input features, $h$ is the number of neurons in the hidden layer, and $W$ is the matrix of weights in the first layer.       
> Note: in PyTorch, the matrix of weights is given in transposed form. 

In order for the model to generalize well, the last quantity ($\text{Grad}$) should be prevented from becoming too large. We rely instead on the inequality $\text{Grad}\leq \|W\|_\text{Frob}$​, that we can prove for this model, and will try to control the weight norms instead. 

- **Task (2.b.1)** **(10 pts.)** Implement the function `Frob(nn)` that computes $\|W\|_\text{Frob}$​, i.e., the Froebinius norm of the model weights.
- **Task (2.b.1)** **(10 pts.)** Implement the function `Grad(nn,X)`. This takes as input the neural network and some dataset, and returns the averaged gradient norm ($\text{Grad}$)
- **Task (2.b.c)** **(5 pts.)** Run the cell below. Can you describe what you observe and why? Please refer to excercise 2 from the theory part.

In [ ]:
def Frob(nn):
    # ----------------------------------
    #  Your code here
    # ----------------------------------
    pass
    
    
def Grad(nn,X):
    # ----------------------------------
    #  Your code here
    # ----------------------------------
    pass

In [ ]:
# The following code measures these three quantities before and after training the model
# N.B. No regularization enforced here. 

def full_logging(name,nn):
    return logging(name,nn) + ' | WFrob: %7.3f | Grad: %7.3f'%(Frob(nn),Grad(nn,Xtest))

nnr = NeuralNetworkRegressor()
print(full_logging('Before',nnr))
nnr.fit(Xtrain,Ttrain)
print(full_logging('After',nnr)) 

#### **Task (2.c)** **(15 pts.)** 

Consider a new objective $\mathcal{L}_{Frob}(θ)=MSE(θ)+\lambda⋅\|W\|_\text{Frob}^2$​. The first term is the original mean square error objective (see above) and  the second term is the regularization term. We hardcode the penalty coeffecient to $\lambda=0.01$. A downside of the Frobenius norm is that it is not a very tight upper bound of the gradient, that is, penalizing it is does not penalize specifically high gradient. Instead, other useful properties of the model could be negatively affected by it.


- **Task 2.c.1** **(10 pts.)**

    Create a new regressor by reimplementing the regularization function with the Frobenius norm regularizer. You may for this task call the norm functions implemented in the question above, but this time you also need to ensure that these functions can be differentiated with respect to the weight parameters (i.e., make sure that gradients can be backpropagated through the model).

- **Task 2.c.2** **(5 pts.)**       

    What do you observe? What if you change the value of $\lambda$? What can you conclude in general and how the new regressor performs compared to the old one?

In [ ]:
class FrobRegressor(NeuralNetworkRegressor):
    
    def reg(self):
        # ----------------------------------
        #  Your code here
        # ----------------------------------
        pass

In [ ]:
nnfrob = FrobRegressor()
nnfrob.fit(Xtrain,Ttrain)

print(full_logging('NN',nnr))
print(full_logging('NN+Frob',nnfrob))